In [ ]:
import pandas as pd
import keras
from keras.layers import Input, Dense, LSTM, Conv1D, \
    BatchNormalization, Dropout, MaxPooling1D, Flatten
import tensorflow as tf
import matplotlib.pyplot as plt
from Indicator import DataManage
from DataScaler import *
from tensorflow.keras.utils import to_categorical
data = pd.read_csv("testlabel.csv",index_col=0)
data.head()


In [ ]:
def checking_labeling(labeling_data):
    step = 1000
    for i in range(0, len(labeling_data), step):
        name = str(labeling_data.iloc[i]["datetime"])+ " ~ " + str(labeling_data.iloc[i + step - 1]["datetime"])
        fig = plt.figure(figsize=(16,9))
        fig = plt.title(name)
        fig = plt.plot(labeling_data.iloc[i:i + step]["datetime"], labeling_data.iloc[i:i + step]["close"])
        for j in range(i, i + step):
            if labeling_data.iloc[j]["label"] == 0:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "r")
            if labeling_data.iloc[j]["label"] == 1:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "b")
        
        plt.savefig("Labeling_picture/" + name + ".png")

In [ ]:
parameter = [
        {"rsi" : {"period" : 14}},
        {"ma" : {"period" : 7}},
        {"ma" : {"period" : 25}},
        {"ema" :{"period" : 7}},
        {"ema" :{"period" : 25}},
        {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 21,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 12, "slow_period" : 26}}
]
DataManageBot = DataManage(data, parameter = parameter)
data = DataManageBot.get_data()
data.head()

In [ ]:
# test_size = int(len(df) * 0.2)
test_size = 1440 * 30
X,Y = data.drop(['label','datetime'],axis = 1),data['label']
X = Data_StandardScaler(X)

x_train = X[:-test_size]
y_train = Y[:-test_size]
x_test = X[-test_size:]
y_test = Y[-test_size:]
x_train.shape,y_train.shape

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(256,input_shape = (x_train.shape[1],)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(64,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(32,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(3,activation="softmax"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    metrics=["accuracy"],
                    loss = tf.keras.losses.CategoricalCrossentropy())

In [ ]:
model.fit(x_train,to_categorical(y_train),epochs=10)

In [7]:
pred = model.predict(x_test)
pred[:10]

array([[1.2016799e-03, 1.2838879e-01, 8.7040949e-01],
       [1.2195334e-03, 1.2816569e-01, 8.7061471e-01],
       [1.1050779e-03, 1.2930144e-01, 8.6959344e-01],
       [1.0186307e-03, 1.2930706e-01, 8.6967432e-01],
       [8.0599834e-04, 1.3112731e-01, 8.6806673e-01],
       [1.3810067e-03, 1.2695873e-01, 8.7166023e-01],
       [1.2481364e-03, 1.2763278e-01, 8.7111914e-01],
       [2.1066677e-03, 1.2337117e-01, 8.7452215e-01],
       [1.8297404e-03, 1.2472620e-01, 8.7344414e-01],
       [2.3191124e-03, 1.1256401e-01, 8.8511693e-01]], dtype=float32)

In [8]:
result_df = pd.DataFrame(pred[:,:2],columns = ["low","high"])
result_df

,low,high
0,0.001202,0.128389
1,0.001220,0.128166
2,0.001105,0.129301
3,0.001019,0.129307
4,0.000806,0.131127
...,...,...
43195,0.041896,0.001460
43196,0.050888,0.001128
43197,0.031662,0.003457
43198,0.033559,0.002840


In [9]:
result_df.describe()

,low,high
count,43200.000000,4.320000e+04
mean,0.031651,3.629267e-02
std,0.041537,4.353641e-02
min,0.000004,2.402711e-08
25%,0.002899,9.410837e-04
50%,0.009892,1.406131e-02
75%,0.044803,6.520302e-02
max,0.275973,1.701701e-01


In [10]:
def percent(data,per):
    high = data.high.tolist()
    low = data.low.to_list()
    high.sort()
    low.sort()
    entire_len = len(high)/100
    
    return high[int(entire_len * per)], low[int(entire_len * per)]

In [18]:
high_target,low_target = percent(result_df,99)

In [19]:
result_label = []
cnt = 0
for i in range(len(result_df)):
    if result_df.iloc[i]["low"] > low_target:
        result_label.append(0)
    elif result_df.iloc[i]["high"] > high_target:
        result_label.append(1)
    else:
        result_label.append(-1)
        cnt+=1
cnt

42338

In [20]:
ck = 0 # 정답 카운트
cnt = 0 # 총 카운트
report_list = []
report_pred = []
for i in range(len(result_label)):
    if result_label[i] != -1:
        if result_label[i] == y_test.iloc[i]:
            ck += 1
        cnt += 1
print("accuracy :",round(ck/cnt * 100,2),"%")

accuracy : 24.48 %


In [21]:
Data = data.iloc[-test_size:].drop(["datetime", "label"],axis = 1)
Data

,open,high,low,close,volume,rsi_14,mean_7,mean_25,ema_7,ema_25,...,fast_d,slow_k,slow_d,BBL,BBM,BBU,BBP,K_9_3,D_9_3,J_9_3
2670188,23492.63,23499.13,23486.63,23493.90,147.02654,66.289732,23489.815714,23460.8492,23489.524825,23467.037020,...,72.606665,72.606665,76.950331,23418.273425,23418.273425,23418.273425,23418.273425,66.983856,69.644851,61.661865
2670189,23493.92,23499.12,23490.20,23494.48,155.32715,66.510617,23494.140000,23463.5844,23490.763619,23469.148019,...,69.612267,69.612267,74.504309,23421.946813,23421.946813,23421.946813,23421.946813,65.863728,68.384477,60.822229
2670190,23494.92,23511.74,23492.18,23503.32,314.32097,69.762661,23497.898571,23466.2016,23493.902714,23471.776633,...,72.839662,72.839662,73.949427,23427.024664,23427.024664,23427.024664,23427.024664,69.801099,68.856684,71.689928
2670191,23503.73,23508.38,23500.00,23506.66,173.76635,70.911984,23498.874286,23469.1060,23497.092036,23474.459969,...,76.964904,76.964904,74.954586,23434.440123,23434.440123,23434.440123,23434.440123,74.553943,70.755771,82.150289
2670192,23505.99,23519.44,23504.79,23519.44,258.85883,74.851006,23501.320000,23472.6384,23502.679027,23477.919971,...,84.643269,84.643269,78.184147,23439.773839,23439.773839,23439.773839,23439.773839,83.035962,74.849168,99.409551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713383,22407.52,22409.90,22404.17,22406.38,34.44410,48.081544,22405.164286,22410.4920,22407.035342,22408.112195,...,52.038974,52.038974,44.476150,22399.116351,22399.116351,22399.116351,22399.116351,58.146434,50.956732,72.525838
2713384,22406.03,22407.73,22397.90,22400.81,70.36613,42.576564,22405.144286,22410.2624,22405.479006,22407.550488,...,44.109398,44.109398,44.353899,22398.149859,22398.149859,22398.149859,22398.149859,49.083875,50.332446,46.586733
2713385,22401.23,22407.14,22399.70,22405.29,42.83763,47.757506,22406.562857,22410.0708,22405.431755,22407.376604,...,46.121808,46.121808,44.943202,22398.056941,22398.056941,22398.056941,22398.056941,51.040741,50.568544,51.985134
2713386,22405.71,22407.75,22402.16,22404.14,41.45687,46.595344,22405.802857,22409.4768,22405.108816,22407.127635,...,45.677403,45.677403,45.187936,22398.649087,22398.649087,22398.649087,22398.649087,50.292113,50.476401,49.923538


In [22]:
import matplotlib.pyplot as plt
def checking_labeling(labeling_data):
    step = 1000
    for i in range(0, len(labeling_data) - step, step):
        name = str(labeling_data.iloc[i]["datetime"])+ " ~ " + str(labeling_data.iloc[i + step - 1]["datetime"])
        fig = plt.figure(figsize=(16,9))
        fig = plt.title(name)
        fig = plt.plot(labeling_data.iloc[i:i + step]["datetime"], labeling_data.iloc[i:i + step]["close"])
        for j in range(i, i + step):
            if labeling_data.iloc[j]["result_label"] == 0:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "r")
            if labeling_data.iloc[j]["result_label"] == 1:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "b")
        
        plt.savefig("Test_Result/" + name + ".png")
        plt.close()

In [23]:
result_data = pd.concat([data.iloc[-test_size:].reset_index(drop = True), pd.DataFrame(result_label,columns=["result_label"])],axis=1)
result_data.head()

,datetime,open,high,low,close,volume,label,rsi_14,mean_7,mean_25,...,slow_k,slow_d,BBL,BBM,BBU,BBP,K_9_3,D_9_3,J_9_3,result_label
0,2023-02-03 11:36:00,23492.63,23499.13,23486.63,23493.90,147.02654,2,66.289732,23489.815714,23460.8492,...,72.606665,76.950331,23418.273425,23418.273425,23418.273425,23418.273425,66.983856,69.644851,61.661865,-1
1,2023-02-03 11:37:00,23493.92,23499.12,23490.20,23494.48,155.32715,2,66.510617,23494.140000,23463.5844,...,69.612267,74.504309,23421.946813,23421.946813,23421.946813,23421.946813,65.863728,68.384477,60.822229,-1
2,2023-02-03 11:38:00,23494.92,23511.74,23492.18,23503.32,314.32097,2,69.762661,23497.898571,23466.2016,...,72.839662,73.949427,23427.024664,23427.024664,23427.024664,23427.024664,69.801099,68.856684,71.689928,-1
3,2023-02-03 11:39:00,23503.73,23508.38,23500.00,23506.66,173.76635,2,70.911984,23498.874286,23469.1060,...,76.964904,74.954586,23434.440123,23434.440123,23434.440123,23434.440123,74.553943,70.755771,82.150289,-1
4,2023-02-03 11:40:00,23505.99,23519.44,23504.79,23519.44,258.85883,2,74.851006,23501.320000,23472.6384,...,84.643269,78.184147,23439.773839,23439.773839,23439.773839,23439.773839,83.035962,74.849168,99.409551,-1


In [24]:
checking_labeling(result_data)

In [ ]:
X,Y = data.drop(['label','datetime'],axis = 1),data['label']
test_size = 1440 * 30
x_train = X[:-test_size]
y_train = Y[:-test_size]
x_test = X[-test_size:]
y_test = Y[-test_size:]
x_test['label'] = result_label
from tqdm import tqdm
set_amount = 0.010
time_window = 1
fee = 0.0008
max_buy = 100000

buy_flag = 0
sell_flag = 0
for k in range(0,1):
    buy_count = 0
    buy_price = []
    result = 0
    result_list = []
    total = 0
    win = 0
    buy_count_list = []
    amount = 0.000
    average_risk = []
    average_price = 0
    trading = []
    buy_index = []

    for i in tqdm(range(10,len(x_test)-1)):
        if x_test.iloc[i]['label'] == 0 and buy_count < max_buy and buy_flag >= 20:
            buy_count += 1
            average_price = ( average_price * amount + x_test.iloc[i]['close'] * set_amount) / (amount + set_amount)
            amount += set_amount
            buy_price.append(x_test.iloc[i]['close'])
            buy_index.append(i)
        elif x_test.iloc[i]['label'] == 0:
            buy_flag += 1

        if x_test.iloc[i-1]['label'] == 1 and buy_count != 0 and sell_flag >= 10:
            temp = (x_test.iloc[i]['close']-average_price) * amount - (average_price * fee * amount)
            line = [temp,x_test.iloc[i]['close'],average_price,amount,i]
            buy_count_list.append(buy_count)
            if temp > 0:
                win += 1
            average_price = 0
            total += 1
            result += temp
            buy_count = 0
            amount = 0.000
            buy_price.clear()
            trading.append(line)
            buy_flag = 0
            sell_flag = 0

        elif x_test.iloc[i-1]['label'] == 1:
            sell_flag += 1
            buy_flag = 0

        if buy_count > 0:
            average_risk.append(round((sum(buy_price)/buy_count-x_test.iloc[i]['close'])/x_test.iloc[i]['close'],2))
        result_list.append(result)
    buy_count_list.sort()
    average_risk.sort()
    
    try:
        print('평균 매매 횟수',sum(buy_count_list)/len(buy_count_list))
        print('이익',result)
        print('승률',win/total)
        print("리스크",average_risk[-1])
        print("최대 보유",buy_count_list[-1])
        print("트레이딩 횟수:",total)
    except:
        print("ZeroDivisionError ")

In [ ]:
import mplfinance as mpf
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("cobok"))))
from Data.DB_Manage import DB_Bot

data = pd.read_csv("BTC_USDT_1m.csv",index_col=0)
data.head()

In [ ]:
data.datetime = pd.to_datetime(list(data.datetime))

In [ ]:
data.columns = ["datetime", "Open", "High", "Low", "Close", "Volume"]
data = data.set_index("datetime")
data.head()

In [ ]:
mpf.plot(data.iloc[:1000])
mpf.plot(data.iloc[1100:1200])

In [ ]:
ap = mpf.make_addplot(data.iloc[200:300])
mpf.plot( data.iloc[:100], addplot=ap)